In [1]:
import pandas as pd
import numpy as np

In [2]:
# df=pd.read_csv('./segmentation.data')
### Generate Clusters
from sklearn import cluster, datasets
noisy_moons = datasets.make_moons(n_samples=100, noise=.05)
df = pd.DataFrame(noisy_moons[0]) ## convert to DF

In [3]:
# Visualize the data
import matplotlib.pyplot as plt
plt.scatter(df[0],df[1])
plt.show()

In [4]:
def find_distance_matrix(data):
    euclid_distance = []
    for i in data:
        distance = []
        for j in data:
            distance.append(np.linalg.norm(i - j) * np.linalg.norm(i - j))
        distance = np.array(distance)
        euclid_distance.append(distance)
    euclid_distance = np.array(euclid_distance)
    return euclid_distance

def inverse_squareform(matrix):
    inv_sqfrm = []
    for i in range(len(matrix)):
        for j in range(i+1, len(matrix[i])):
            inv_sqfrm.append(matrix[i][j])
    inv_sqfrm = np.array(inv_sqfrm)
    return inv_sqfrm

def rbfkernel(gamma, data):
    return np.exp(-gamma * distance)

data = np.array(df)

In [5]:
# Making kernel matrix
distance = find_distance_matrix(data)
gamma = 1/(2*np.var(inverse_squareform(distance)))
kernel = rbfkernel(gamma, distance)

In [6]:
W=kernel[:]
cutoff = 0.980
for i in range(len(kernel)):
    for j in range(len(kernel)):
        if kernel[i][j] < cutoff:
            W[i][j] = 0

In [7]:
# Making degree matrix
D = np.zeros(shape=(len(W),len(W)))
for i in range(len(W)):
    D[i][i] = np.sum(W[i])           

In [8]:
L=D-W
## Find eign values, divide vecotrs by root lamba, sort ,pick k highest
tmp=np.linalg.eig(L) ## Using Centred Matrix to find eign values
eign_values=tmp[0]
eign_vector=tmp[1]
indexes = eign_values.argsort()[::1]
second_vector = eign_vector[:, indexes[1]]

In [9]:
plt.plot(np.sort(eign_values),'ro')
plt.show()

In [72]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2).fit(second_vector.reshape(-1,1))
label=kmeans.labels_

In [73]:
plt.scatter([data[i][0] for i in range(len(data)) if label[i] == 1], [data[i][1] for i in range(len(data)) if label[i] == 1], color='green', alpha=0.5)
plt.scatter([data[i][0] for i in range(len(data)) if label[i] == 0], [data[i][1] for i in range(len(data)) if label[i] == 0], color='red', alpha=0.5)
plt.show()

In [50]:
################
################ Self Implemented K means
################
# k=2
# import random
# def KMeans_implementation(data):
#     alpha = np.zeros(shape = (len(data), k))

#     random.seed()
#     indexes = random.sample(range(0, len(data)), k)
#     means = data[indexes]

#     for i in range(15):
#         new_alpha = np.zeros(shape = (len(data),k))
#         for i in range(len(data)):
#             min_value = 123213123123123
#             min_index = -1
#             for j in range(k):
#                 distance = np.linalg.norm(data[i] - means[j])**2
#                 if distance < min_value:
#                     min_value = distance
#                     min_index = j
#             new_alpha[i][min_index] = 1
#         alpha = new_alpha[:]
#         class_frequency = pd.DataFrame(alpha)    
#         cluster_count = [class_frequency[i].value_counts()[1] for i in range(k)]
#         for j in range(k):
#             means[j] = 0
#             for i in range(len(data)):
#                 means[j] += (alpha[i][j] * data[i])
#             means[j] = means[j] / float(cluster_count[j])

#     labels = []
#     for i in range(len(alpha)):
#         for j in range(k):
#             if alpha[i][j] == 1:
#                 labels.append(j)
#     return labels

# labels = KMeans_implementation(second_vector)